In [1]:
"""
!pip install llama_index
!pip install llama-index-utils-workflow
!pip install llama-index-core
!pip install llama-index-retrievers-bm25
!pip install llama-index-llms-azure-openai
"""

'\n!pip install llama_index\n!pip install llama-index-utils-workflow\n!pip install llama-index-core\n!pip install llama-index-retrievers-bm25\n!pip install llama-index-llms-azure-openai\n'

## Setting up calls to OpenAI

In [2]:
import sys
import os 
import nest_asyncio

# Sanity check
print(sys.executable)
nest_asyncio.apply()


"""
OPENAI_API_MODEL_NAME = "gpt-4o"
OPENAI_DEV_KEY = "***************"
OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = "2023-03-15-preview"
OPENAI_API_BASE = "https://*******.openai.azure.com/"
OPENAI_MODEL_EMBEDDING = "text-embedding-ada-002"
"""

from dotenv import load_dotenv
load_dotenv() 

"""
OPENAI_API_MODEL_NAME = os.environ["OPENAI_API_MODEL_NAME"]
OPENAI_DEV_KEY = os.environ["OPENAI_DEV_KEY"]
OPENAI_API_TYPE = os.environ["OPENAI_API_TYPE"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]
OPENAI_API_BASE = os.environ["OPENAI_API_BASE"]
OPENAI_MODEL_EMBEDDING = os.environ["OPENAI_MODEL_EMBEDDING"]
"""



/Users/amorvan/Documents/code_dw/llm_collection/.venv/bin/python


'\nOPENAI_API_MODEL_NAME = os.environ["OPENAI_API_MODEL_NAME"]\nOPENAI_DEV_KEY = os.environ["OPENAI_DEV_KEY"]\nOPENAI_API_TYPE = os.environ["OPENAI_API_TYPE"]\nOPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]\nOPENAI_API_BASE = os.environ["OPENAI_API_BASE"]\nOPENAI_MODEL_EMBEDDING = os.environ["OPENAI_MODEL_EMBEDDING"]\n'

In [3]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.llms.openai import OpenAI
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings





In [4]:
# Be careful to not load
llm = OpenAI(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])


In [5]:
response = llm.complete("Paul Graham is ")
print(response)

Paul Graham is a prominent computer scientist, entrepreneur, and venture capitalist, best known for his work in the field of programming languages and for co-founding the startup accelerator Y Combinator. He is also recognized for his essays on technology, startups, and entrepreneurship, which have gained a significant following in the tech community. Graham co-authored the programming language Lisp dialect called Arc and has been influential in the development of various web technologies. His insights on startups and innovation have made him a respected figure in Silicon Valley.


## Your first Workflow

In [6]:
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    Context,
StartEvent, StopEvent
)


class MyWorkflow(Workflow):

    @step
    async def do_it(self, ev: StartEvent) -> StopEvent:
        return StopEvent(result="coucou 2")
        



In [7]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(MyWorkflow, filename="basic_workflow.html")

<class 'NoneType'>
<class 'llama_index.core.workflow.events.StopEvent'>
basic_workflow.html


In [8]:
w = MyWorkflow()

r = await w.run()

In [9]:
r

'coucou 2'

## Exercise : 

### Make a workflow that answers the query of a user

User : What is an Ipod

```python
w = MyCustomWorkflow()
result = await w.run()
print(result)
```
"An ipod is ...."


In [10]:

class MyCustomWorkflow(Workflow):

    @step
    async def answer(self, ev: StartEvent) -> StopEvent:
        question = ev.message
        answer = llm.complete(f"Answer this user question : {question}")
        return StopEvent(result=answer)
        



In [11]:
user_query = "What is an Ipod"

w = MyCustomWorkflow()
result = await w.run(message=user_query)
print(result)

An iPod is a portable media player designed and marketed by Apple Inc. It was first introduced in 2001 and became popular for its ability to store and play music, as well as other media such as videos and podcasts. Over the years, several models of the iPod have been released, including the iPod Classic, iPod Mini, iPod Nano, iPod Shuffle, and iPod Touch. 

The iPod Touch, in particular, functions similarly to an iPhone but without cellular capabilities, allowing users to access apps, browse the internet, and play games in addition to listening to music. The iPod line has significantly influenced the way people consume music and has played a key role in the rise of digital music. However, with the advent of smartphones and streaming services, the popularity of standalone iPods has declined, and Apple officially discontinued the iPod Touch in 2022.


### A more complex worflow

Answer in two steps : 
- Find the answer of the question
- Transform it into a poem

Use a prompt template for it.


In [12]:
from llama_index.core import PromptTemplate


class AnswerEvent(Event):
    pass



class MyNewWorkflow(Workflow):

    @step
    async def step_a(self, ev: StartEvent) -> AnswerEvent:
        question = ev.message
        answer = llm.complete(f"Answer this user question : {question}")
        return AnswerEvent(result=answer)


    @step
    async def step_b(self, ev: AnswerEvent) -> StopEvent:
        result = ev.result
        poem = llm.complete(f"Make this text a poem :  {result}")
        return StopEvent(result=poem)
        
        



In [13]:
user_query = "What is an Ipod"

w = MyNewWorkflow()
result = await w.run(message=user_query)
print(result)

In two thousand one, a marvel was born,  
An iPod emerged, a new age was sworn.  
A portable player, sleek and refined,  
By Apple Inc., a treasure to find.  

With music and videos, it captured the heart,  
A world of sound, a digital art.  
From Classic to Nano, each model a gem,  
A journey through time, a musical hymn.  

The Classic held memories, a hard drive so vast,  
While Nano was nimble, a flash player cast.  
The Shuffle, so compact, with no screen to see,  
And Touch, like an iPhone, with apps set free.  

Though smartphones now reign, the iPod still glows,  
A chapter in history, where nostalgia flows.  
In twenty-two, the Touch took its bow,  
The end of an era, we cherish it now.  

So here’s to the iPod, a legend in sound,  
In the heart of our music, its spirit is found.  
Though times may have changed, and new paths we tread,  
The echoes of iPods will never be dead.  
